### Ce programme sert a creer la tables historical_games et historical_games_details en format csv qui pourra par la suite etre manuellement chargee dans GCP.

In [1]:
import requests
import simplejson as json
from datetime import date, timedelta
import datetime
import sys

In [2]:
def get_schedule(date):
    response = requests.get('https://statsapi.web.nhl.com/api/v1/schedule?date={}'.format(date))
    return response.json()

def fetch_and_write(date):
    hockey = get_schedule(date)
    if hockey['totalItems'] != 0:
        hockey = hockey['dates'][0]['games']
        game_list = []
        for games in hockey:
            game_id = str(games['gamePk'])
            game_date = str(games['gameDate'][0:10])
            game_type = games['gameType']
            season = str(games['season'])
            for team in games['teams']:
                if team=='home':
                    home_away=str(1)
                else:
                    home_away=str(0)
                team_id = str(games['teams'][team]['team']['id'])
                team_score = str(games['teams'][team]['score'])
                string = game_id + ',' + game_date + ',' + game_type + ',' + season + ',' + home_away + ',' + team_id + ',' + team_score
                game_list.append(string)

        print('\n'.join(game_list))
    return

In [3]:
sys.stdout = open('historical-games-20080601-20200801', 'w')

start_date = datetime.date(2016, 9, 30)
end_date = datetime.date(2020, 8, 1)

delta = datetime.timedelta(days=1)

while start_date <= end_date:
    extract = fetch_and_write(start_date.strftime("%Y-%m-%d"))
    start_date += delta